In [20]:
import os
import keras.models as models
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import LeakyReLU, Input, Add
from keras import backend as K
import cv2
import numpy as np
import json
from __future__ import absolute_import
from __future__ import print_function
import itertools
from helper import *


DataPath = './CamVid/'
data_shape = 360,480


D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [53]:
def load_data(mode):
    data = []
    label = []
    with open(DataPath + mode +'.txt') as f:
        txt = f.readlines()
        txt = [line.split(' ') for line in txt]
    for i in range(len(txt)):
        data.append(normalized(cv2.imread(os.getcwd() + txt[i][0][7:])))
        label.append(np.resize(cv2.imread(os.getcwd() + '/CamVid/LabeledApproved_full' + txt[i][0][20:][:-4] + '_L.png'),(360,480,3)))
    return np.array(data), np.array(label)

train_data, train_label = load_data("train")
train_label = np.reshape(train_label,(367,360,480,3))


In [54]:
test_data, test_label = load_data("test")
test_label = np.reshape(test_label,(233,360,480,3))

In [57]:
val_data, val_label = load_data("val")
val_label = np.reshape(val_label,(101,360,480,3))

In [58]:
def SegnetGen(input, kernel, filter_size, pad, pool_size):
    
    Generator = input

    a = ZeroPadding2D(padding=(pad,pad))(Generator)    
    conv1 = Convolution2D(filter_size, kernel, kernel, border_mode='valid')(a)
    conv1 = BatchNormalization()(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv1)
    conv1 = ZeroPadding2D(padding=(pad,pad))(conv1)  
    conv2 = Convolution2D(128, kernel, kernel, border_mode='valid')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = LeakyReLU(alpha=0.2)(conv2)
    conv2p = MaxPooling2D(pool_size=(pool_size, pool_size))(conv2)
    conv2p = ZeroPadding2D(padding=(pad,pad))(conv2p)
    conv3 = Convolution2D(256, kernel, kernel, border_mode='valid')(conv2p)
    conv3 = BatchNormalization()(conv3)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv3p = MaxPooling2D(pool_size=(pool_size, pool_size))(conv3)
    conv3p = ZeroPadding2D(padding=(pad,pad))(conv3p)
    conv4 = Convolution2D(512, kernel, kernel, border_mode='valid')(conv3p)
    conv4 = BatchNormalization()(conv4)
    Generator2 = LeakyReLU(alpha=0.2)(conv4)
    b = ZeroPadding2D(padding=(pad,pad))(Generator2)
    conv5 = Convolution2D(512, kernel, kernel, border_mode='valid')(b)
    conv5 = BatchNormalization()(conv5)
    conv5 = UpSampling2D(size=(pool_size,pool_size))(conv5)
    conv6 = ZeroPadding2D(padding=(pad,pad))(conv5)
    conv6 = Convolution2D(256, kernel, kernel, border_mode='valid')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Add()([conv3,conv6])
    conv6 = UpSampling2D(size=(pool_size,pool_size))(conv6)
    conv6 = ZeroPadding2D(padding=(pad,pad))(conv6) 
    conv7 = Convolution2D(128, kernel, kernel, border_mode='valid')(conv6)
    conv7 = BatchNormalization()(conv7)
    conv7 = Add()([conv2,conv7])
    conv7 = UpSampling2D(size=(pool_size,pool_size))(conv7)
    conv8 = ZeroPadding2D(padding=(pad,pad))(conv7)
    conv8 = Convolution2D(filter_size, kernel, kernel, border_mode='valid')(conv8)
    conv8 = BatchNormalization()(conv8)
    Generator4 = (Convolution2D(3, 1, 1, border_mode='valid',))(conv8)
    g_out = (Activation('softmax'))(Generator4)
    gen = Model(input = Generator, output = g_out)
    gen.compile(loss="categorical_crossentropy", optimizer='adadelta', metrics=["accuracy"])
    
    return gen, g_out


In [72]:
def discriminatorGen(input, kernel, filter_size, pad, pool_size):
    
    Discriminate = input
    
    a = ZeroPadding2D(padding=(pad,pad))(Discriminate)
    conv1 = Convolution2D(filter_size, kernel, kernel, border_mode='valid')(a)
    conv1 = BatchNormalization()(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv1)
    conv1 = ZeroPadding2D(padding=(pad,pad))(conv1)
    conv2 = Convolution2D(128, kernel, kernel, border_mode='valid')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = LeakyReLU(alpha=0.2)(conv2)
    conv2p = MaxPooling2D(pool_size=(pool_size, pool_size))(conv2)
    conv2p = ZeroPadding2D(padding=(pad,pad))(conv2p)
    conv3 = Convolution2D(64, kernel, kernel, border_mode='valid')(conv2p)
    conv3 = BatchNormalization()(conv3)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv3p = MaxPooling2D(pool_size=(pool_size, pool_size))(conv3)
    conv3p = ZeroPadding2D(padding=(pad,pad))(conv3p)
    conv4 = Convolution2D(32, kernel, kernel, border_mode='valid')(conv3p)
    conv4 = BatchNormalization()(conv4)
    conv4 = LeakyReLU(alpha=0.2)(conv4)
    out = Flatten()(conv4)
    out = Dense(2)(out)
    d_out = Activation('sigmoid')(out)
    discrim = Model(input = Discriminate, output = out)
    discrim.compile(loss="binary_crossentropy", optimizer='adadelta', metrics=["accuracy"])
    
    return discrim, d_out
    
    

In [117]:
def GanGen(Gan_in, g_out, G, D):
    z = G(Gan_in)
    Gan_out = D(z)
    GAN = Model(inputs=[Gan_in, g_out],outputs=Gan_out)
    GAN.compile(loss="binary_crossentropy", optimizer='adadelta', metrics=["accuracy"])
    
    return(GAN, Gan_out)
    

In [60]:
nb_epochs = 3
batch_size = 2

kernel = 3
filter_size = 64
pad = 1
pool_size = 2

In [61]:
# Generator Calls:
g_input = Input(shape=(360,480,3))
G, g_out = SegnetGen(g_input, kernel, filter_size, pad, pool_size)
G.summary()

D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
  import sys
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="valid")`
  del sys.path[0]
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="valid")`
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="valid")`
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="valid")`
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 360, 480, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 362, 482, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 360, 480, 64) 1792        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 360, 480, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_l

D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (1, 1), padding="valid")`
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:58: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`


In [85]:
d_input = Input(shape=(360,480,12))
D, d_out = discriminatorGen(d_input, kernel, filter_size, pad, pool_size)
D.summary()



D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
  import sys
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="valid")`
  del sys.path[0]
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 360, 480, 12)      0         
_________________________________________________________________
zero_padding2d_61 (ZeroPaddi (None, 362, 482, 12)      0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 360, 480, 64)      6976      
_________________________________________________________________
batch_normalization_61 (Batc (None, 360, 480, 64)      256       
_________________________________________________________________
leaky_re_lu_45 (LeakyReLU)   (None, 360, 480, 64)      0         
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 180, 240, 64)      0         
_________________________________________________________________
zero_padding2d_62 (ZeroPaddi (None, 182, 242, 64)      0         
__________

D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid")`
D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [118]:
GAN_in = Input(shape=(360,480,3))
g_out = Input(shape=(360,480,12))
GAN, GAN_out = GanGen(GAN_in,g_out, G, D)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        (None, 360, 480, 3)       0         
_________________________________________________________________
model_17 (Model)             (None, 360, 480, 12)      5467980   
_________________________________________________________________
model_16 (Model)             (None, 2)                 347042    
Total params: 5,815,022
Trainable params: 5,810,606
Non-trainable params: 4,416
_________________________________________________________________


In [133]:
nb_epochs = 500
batch_size = 5

kernel = 3
filter_size = 64
pad = 1
pool_size = 2

def train(GAN, G, D, batch_size, kernel, filter_size, pad, pool_size, nb_epochs,
          train_data, train_label, test_data, test_label):
    d_loss = []
    g_loss = []
    e_range = range(nb_epochs)
    for epoch in e_range:
        sample = np.random.randint(0,train_data.shape[0], batch_size)
        g_out = G.predict(train_data[sample])
        x = np.concatenate((train_label[sample], g_out))
        y = np.zeros((2*batch_size, 2))
        y[:batch_size, 1] = 1
        y[batch_size:, 0] = 1
        
        D.trainable = True
        for layer in D.layers:
            layer.trainable = True
        
        d_loss.append(D.train_on_batch(x,y))
        D.trainable = False
        for layer in D.layers:
            layer.trainable = False        
        z = np.zeros((batch_size,2))
        z[:batch_size, 1] = 1 
        g_loss.append(GAN.train_on_batch([train_data[sample],train_label[sample]],z))
        print("Epoch #{}: Generative Loss: {}, Discriminative Loss: {}".format(
            epoch + 1, g_loss[-1], d_loss[-1]))

    return G, g_out, x, y, d_loss, g_loss, z 

G, g_out, x, y, d_loss, g_loss, z  = train(GAN, G, D, batch_size, kernel, filter_size,
                                           pad, pool_size, nb_epochs, train_data,
                                           train_label, test_data, test_label)

D:\Apps\Anaconda\envs\tensorflow\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch #1: Generative Loss: [1.6118095, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #2: Generative Loss: [0.012216089, 0.1], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #3: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #4: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #5: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #6: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #7: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #8: Generative Loss: [1.5942385, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #9: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #10: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #11: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #

Epoch #91: Generative Loss: [1.5942385, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #92: Generative Loss: [1.5942385, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #93: Generative Loss: [3.188477, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #94: Generative Loss: [1.5942385, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #95: Generative Loss: [3.188477, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #96: Generative Loss: [3.2060478, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #97: Generative Loss: [1.5942385, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #98: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #99: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #100: Generative Loss: [3.188477, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #101: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #102: Generativ

Epoch #182: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #183: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #184: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #185: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #186: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #187: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #188: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #189: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #190: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #191: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #192: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss

Epoch #271: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #272: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #273: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #274: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #275: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #276: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #277: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #278: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #279: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #280: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #281: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss

Epoch #360: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #361: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #362: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #363: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #364: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #365: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #366: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #367: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #368: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #369: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #370: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss

Epoch #449: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #450: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #451: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #452: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #453: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #454: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #455: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #456: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #457: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #458: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss: [1.0960467e-07, 0.0]
Epoch #459: Generative Loss: [1.0960467e-07, 0.0], Discriminative Loss